<a href="https://colab.research.google.com/github/Ajay-Sai-Kiran/Natural-Language-Processing/blob/main/Day_6_Auto_Spelling_Corrector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Auto Spelling corrector

research paper Link: http://static.googleusercontent.com/media/research.google.com/en/us/pubs/archive/36180.pdf

Link for Bigtxt: http://norvig.com/big.txt

#How It Works: Some Probability Theory

The call correction(w) tries to choose the most likely spelling correction for w. There is no way to know for sure (for example, should "lates" be corrected to "late" or "latest" or "lattes" or ...?), which suggests we use probabilities. We are trying to find the correction c, out of all possible candidate corrections, that maximizes the probability that c is the intended correction, given the original word w:



argmaxc ∈ candidates P(c|w)

By Bayes' Theorem this is equivalent to:

argmaxc ∈ candidates P(c) P(w|c) / P(w)

Since P(w) is the same for every possible candidate c, we can factor it out, giving:

argmaxc ∈ candidates P(c) P(w|c)

The four parts of this expression are:

**Selection Mechanism**: argmax
We choose the candidate with the highest combined probability.

**Candidate Model**: c ∈ candidates
This tells us which candidate corrections, c, to consider.

**Language Model**: P(c)
The probability that c appears as a word of English text. For example, occurrences of "the" make up about 7% of English text, so we should have P(the) = 0.07.

**Error Model**: P(w|c)
The probability that w would be typed in a text when the author meant c. For example, P(teh|the) is relatively high, but P(theeexyz|the) would be very low.

In [1]:
import re
from collections import Counter
#Language Model
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('/content/big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N
#Error Model
def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)#Selection Mechanism

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)
#Candidate Model
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [3]:
correction('speling')


'spelling'

In [4]:
correction('Apple')

'apple'

In [5]:
correction("guttanberg")

'gutenberg'

In [7]:
correction("whael")

'wheel'

In [8]:
correction('whalle')

'whale'

#Acknowledgments
Peter Norvig